# Assignment 3: Predictive analysis - Part 2: Classification
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

In [1]:
# imports
import pandas as pd
import re

## Introduction

## Dataset Description

**Url:** https://www.kaggle.com/datasets/blastchar/telco-customer-churn <br>
**Name:** Telco Customer Churn <br>
**Author:** The dataset was uploaded to `Kaggle` by a user called `BlastChar`, however `IBM` created the original dataset. <br>
**Purpose:** The `Telco Customer Churn` dataset is a synthetic dataset created by `IBM`. The dataset includes data on 7043 customers of the
fictitious telecommunications company `Telco`. For each customer, data is provided regarding their demographics, their account information, 
and the services they have signed up for. The purpose of the dataset is to use the provided customer data to predict customer churn. A column is provided, 
titled `Churn`, that indicates whether or not the customer left within the last month. Since the churn data is provided, we can easily compare
our classification results against the true values. <br>
**Shape:** There are 7043 rows and 21 columns. (7043, 21)<br>
**Features:** 
- `customerID` (categorical): A unique ID provided to each customer in the form `1234-ABCDE`.
- `gender` (categorical): The gender of the customer (includes `Male` and `Female`).
- `SeniorCitizen` (categorical): Indicates if the customer is a senior.
    - `0`: The customer is not a senior.
    - `1`: The customer is a senior.
- `Partner` (categorical): Indicates if the customer has a partner.
    - `Yes`: The customer has a partner.
    - `No`: The customer does not have a partner. 
- `Dependents` (categorical): Indicates if the customer has dependents.
    - `Yes`: The customer has dependents.
    - `No`: The customer does not have dependents. 
- `tenure` (numerical): The number of months the customer has been with the company [0,72].
- `PhoneService` (categorical): Indicates if the customer has a phone service.
    - `Yes`: The customer has a phone service.
    - `No`: The customer does not have a phone service.
- `MultipleLines` (categorical): Indicates if the customer has multiple lines.
    - `Yes`: The customer has multiple lines.
    - `No`: The customer has one line.
    - `No phone service`: The customer does not have phone service.
- `InternetService` (categorical): The customer's internet connection method.
    - `Fiber optic`: The customer's internet connection uses fiber optic technology.
    - `DSL`: The customer's internet connection uses digital subscriber line (DSL) technology.
    - `No`: The customer does not have internet.
- `OnlineSecurity` (categorical): Indicates if the customer has online security.
    - `Yes`: The customer has online security.
    - `No`: The customer does not have online security.
    - `No internet service`: The customer does not have internet service.
- `OnlineBackup` (categorical): Indicates if the customer has online backup.
    - `Yes`: The customer has online backup.
    - `No`: The customer does not have online backup.
    - `No internet service`: The customer does not have internet service. 
- `DeviceProtection` (categorical): Indicates if the customer has device protection.
    - `Yes`: The customer has device protection.
    - `No`: The customer does not have device protection.
    - `No internet service`: The customer does not have internet service. 
- `TechSupport` (categorical): Indicates if the customer has tech support.
    - `Yes`: The customer has tech support.
    - `No`: The customer does not have tech support.
    - `No internet service`: The customer does not have internet service.
- `StreamingTV` (categorical): Indicates if the customer has streaming TV.
    - `Yes`: The customer has streaming TV.
    - `No`: The customer does not have streaming TV.
    - `No internet service`: The customer does not have internet service.
- `StreamingMovies` (categorical): Indicates if the customer has streaming movies.
    - `Yes`: The customer has streaming movies.
    - `No`: The customer does not have streaming movies.
    - `No internet service`: The customer does not have internet service.
- `Contract` (categorical): The contract term of the customer.
    - `Month-to-month`: The customer pays month-to-month.
    - `One year`: The customer has a one-year contract.
    - `Two year`: The customer has a two-year contract. 
- `PaperlessBilling` (categorical): Indicates if the customer has paperless billing.
    - `Yes`: The customer has paperless billing.
    - `No`: The customer does not have paperless billing.
- `PaymentMethod` (categorical): The customer's payment method.
    - `Electronic check`: The customer pays by electronic check.
    - `Mailed check`: The customer pays by mailed check.
    - `Bank transfer (automatic)`: The customer pays automatically by bank transfer.
    - `Credit card (automatic)`: The customer pays automatically by credit card.
- `MonthlyCharges` (numerical): The amount charged to the customer each month [18.25, 118.75]. 
- `TotalCharges` (numerical): The total amount charged to the customer [0.0, 999.9].
- `Churn` (categorical): Indicates if the customer churned, which means if the customer cancelled their service this month.
    - `Yes`: The customer churned.
    - `No`: The customer did not churn. 

## Loading Data and Basic Exploration

In [2]:
# Read in the dataset from a public repository
url = "https://raw.githubusercontent.com/Natasa127/CSI4142-A3/refs/heads/main/Telco-Customer-Churn.csv"
telco_df = pd.read_csv(url)
telco_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
telco_df.shape

(7043, 21)

## Classification Empirical Study

### (a) Cleaning the data

In this section, we will use the validity checks from `Assignment 2` to determine if the data needs to be cleaned. If needed, an imputation method may be used to clean the data. Note that we will be presenting each of the ten validity checks, although they may be in a different order than in assignment 2. They will be clearly enumerated. Furthermore, in `Assignment 2`, we saved a copy of the invalid rows in our validity checks. In this assignment, we will first check the count of invalid rows; if there are none, we will omit the code that saves a copy, and displays invalid rows. If we find the error count to be non-zero, we will then clean it instead of just saving and displaying the rows.

The professor indicated that it would be sufficient to include one example of each check.

#### Validity Check 1: Exact Duplicates

We will first check for exact duplicates in the dataset. This check verifies that there are no rows that are identical over all columns.

**References:** <br>
Exact Duplicates: https://uottawa.brightspace.com/d2l/le/content/490358/viewContent/6620388/View (Slide 27)

In [5]:
# Exact duplicates check

# Apply the .duplicated method to the DataFrame to create a Series, with exact duplicates set to True
duplicates = telco_df.duplicated()

# Print the number of rows that are exact duplicates
print(f"Number of duplicate rows: {duplicates.sum()}\n")

Number of duplicate rows: 0



Since there are no exact duplicates, no cleaning is required.

#### Validity Check 2: Near Duplicate Errors

In `Assignment 2`, our near duplicate error checker checked if a row differed only by a synonym in a designated attribute. However, when developing our dataset description, we used `.value_counts()` on each categorical attribute, and we noted that there are no synonyms that would cause this error. Thus, we will be using a different type of near duplicate check in this assignment. Instead, we will be checking if any customer's have the exact same information, differing only by their `customerID`.

**References:** <br>
Omit a Column: https://sparkbyexamples.com/pandas/pandas-select-all-columns-except-one-column-in-dataframe/#:~:text=Select%20All%20Except%20One%20Column%20Using%20drop()%20Method%20in,axis%3D1%20or%20columns%20param.

In [6]:
# Near duplicates check

# Apply the .duplcated method to the DataFrame, excluding the customerID column, to create a Series, with exact duplicates of the remaining columns set to True
no_customer_id_duplicates = telco_df.loc[:, telco_df.columns != "customerID"].duplicated()

# Print the number of rows that are near duplicates
print(f"Number of near duplicate rows: {no_customer_id_duplicates.sum()}\n")

Number of near duplicate rows: 22



Since there are 22 near duplicate rows, let us further investigate the actual rows to determine how to handle them.

In [7]:
# Save the invalid rows
invalid_exact_duplicate_df = telco_df.loc[no_customer_id_duplicates]

# Display the first 3 rows that are near duplicates
print("Examples of three near duplicate rows:")
invalid_exact_duplicate_df.head(3)

Examples of three near duplicate rows:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
964,9117-SHLZX,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,45.70,45.7,Yes
1338,1934-SJVJK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes
1491,8605-ITULD,Female,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.55,19.55,No


We believe that despite our efforts to alter our near duplicate checker to identify rows that vary only by `customerID`, these rows **should not be removed.** There are a few reasons for this:

- There is no row that occurs more than 3 times; in essence, there is no row that occurs a completely unreasonable number of times (there is only one row that occurs three times, and the rest occur twice).
- Given that there are 7043 customers, 22 customers having the same information is plausible. There are a couple reasons why.
    - This is a relatively small portion of the overall sample.
    - Unlike the example in the course slides of near duplicates, we do not have a column like `Name` that would be highly likely to be distinct. All of the columns are reasonably likely to overlap.
        - There are not many options for each column. All of the categorical values only provide a few options, and in many cases the same value would occur across multiple columns (ex: `No internet service` would be shared across 6 columns, so it effectively reduces the number of columns that could be a differentiator). Even the numerical columns are not particularly surprising to overlap—since they represent the prices of services, and presumably `Telco` offers a discrete set of services, the minimal duplication is expected.
    - We analyzed all 22 rows that are duplicates, and the values are values that are logically more likely to be repeated; for example, all the customers had a tenure of 1 month, have a month-to-month plan, no dependents, no partner, were (mostly) not seniors, did not have multiple lines, and most of them do not purchase the internet service. Even of those that did purchase the internet service, none of them had any of the packages creating an unlikely combination. These are all relatively common values, and the internet service specifically reduces the number of differentiating columns. Thus, these users with a common service can only be differentiated by demographic information, their payment methods, and their churn, and it is likely that in some cases this would overlap.
        - As an illustrative example, if there were 15 people with a tenure of exactly 42 months, and had a very specific combination of internet packages, then it would be more likely they were duplicates.
        - To analyze the values of near duplicates, change `invalid_exact_duplicate_df.head(3)` to `invalid_exact_duplicate_df.head(22)`. We have left only 3 examples for brevity.

Thus, we have decided not to remove the rows.

#### Validity Check 3: Format errors

We will use the format check from `Assignment 2` to verify if the `customerID` is in the correct format. The only change necessary is to change the regex to verify that the `customerID` is in the format `1234-ABCDE`. None of the other features have a specific format to follow.

**References:** <br>
Regex Rules: https://www.geeksforgeeks.org/perl-regex-cheat-sheet/

In [8]:
# Format check

# Attribute selection
test_attribute = 'customerID'

# We will fix the regex for the customerID format (1234-ABCDE)
format_regex = r"^\d{4}-[A-Z]{5}$"

# Evaluates a single value against a given regex format
def format_filter(value, format_regex):
    return False if not isinstance(value, str) else bool(re.findall(format_regex, value))

# Apply the function to the test attribute, setting invalid formats to True
invalid_format = telco_df[test_attribute].apply(
    lambda attribute: not format_filter(attribute, format_regex)
)

# Print the number of rows with invalid formatting on the chosen test attribute
print(f"Number of rows where the {test_attribute} value has an invalid format: {invalid_format.sum()}\n")

Number of rows where the customerID value has an invalid format: 0



Since there are no rows with an invalid `customerID` format, no cleaning is required.

#### Validity Check 4: Uniqueness errors

We will use the uniqueness check from `Assignment 2` to validate that the `customerID` values are unique. Minimal changes are necessary to the code. We only need to provide the `customerID` feature name, and change the `sales` DataFrame to `telco_df`. None of the other features are intended to be unique.

In [9]:
# Uniqueness check

# Attribute selection
test_attributes = "customerID"

# Store a series of the counts of each value in the chosen column
attribute_series_counts = telco_df[test_attribute].value_counts()

# Evaluates a single value, checking if it is unique in the chosen column
def uniqueness_filter(value, counts):
    if pd.isna(value):
        return False
    if counts[value] == 1:
        return True
    return False

# Apply the function to the test attribute, setting rows with non-unique values in the designated column to True
invalid_uniqueness = telco_df[test_attribute].apply(
    lambda attribute: not uniqueness_filter(attribute, attribute_series_counts)
)

# Print the number of rows with a value that is not unique in the chosen column
print(f"Number of rows where the {test_attribute} value is not unique: {invalid_uniqueness.sum()}\n")

Number of rows where the customerID value is not unique: 0



Since every `customerID` is unique, no cleaning is required.

#### Validity Check 5: Presence check

We will use the presence check from `Assignment 2` to validate that there is no missing information in any of the columns. Minimal changes are necessary to the code. Instead of taking a parameter input, we will simply iterate over all of the columns to determine if any of them are missing values. We also added a check for strings of whitespace.

In [10]:
# Presence check

# Iterate over each column and perform the presence check on each
for column in telco_df:
    # Apply pd.isna() to the test attribute, or check if it is a string of whitespace, setting rows with missing values in the designated column to True
    invalid_presence = telco_df[column].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

    # Print the number of rows with a missing value in the chosen test attribute
    print(f"Number of rows where the {column} value is missing: {invalid_presence.sum()}")

Number of rows where the customerID value is missing: 0
Number of rows where the gender value is missing: 0
Number of rows where the SeniorCitizen value is missing: 0
Number of rows where the Partner value is missing: 0
Number of rows where the Dependents value is missing: 0
Number of rows where the tenure value is missing: 0
Number of rows where the PhoneService value is missing: 0
Number of rows where the MultipleLines value is missing: 0
Number of rows where the InternetService value is missing: 0
Number of rows where the OnlineSecurity value is missing: 0
Number of rows where the OnlineBackup value is missing: 0
Number of rows where the DeviceProtection value is missing: 0
Number of rows where the TechSupport value is missing: 0
Number of rows where the StreamingTV value is missing: 0
Number of rows where the StreamingMovies value is missing: 0
Number of rows where the Contract value is missing: 0
Number of rows where the PaperlessBilling value is missing: 0
Number of rows where th

There are only missing values in the `TotalCharges` feature. We will investigate this further, first by printing out some examples of invalid rows.

In [15]:
# Extract the rows with a missing TotalCharges
invalid_presence = telco_df["TotalCharges"].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

# Save the invalid rows
invalid_presence_df = telco_df.loc[invalid_presence]

# Display the first 3 rows with a missing value
print(f"Examples of three rows where the TotalCharges value is missing:")
invalid_presence_df.head(3)

Examples of three rows where the TotalCharges value is missing:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No


Note that all of the rows that are missing a `TotalCharges` value have a `tenure` of 0. This makes sense; since this is the customer's first month, they have not been charged yet.

In [16]:
telco_df["tenure"].value_counts()[0]

np.int64(11)

There are 11 customers with a `tenure` of zero, corresponding to the number of customers with a missing `TotalCharges` value. Thus, this must be the reason. Let us replace the missing values with a value of 0, as from a logical standpoint, these customers have been charged $0 so far. We will make this change in a copy of the dataset called `cleaned_telco_df`. Any subsequent cleaning will be done using this dataset.

In [17]:
cleaned_telco_df = telco_df.copy()
cleaned_telco_df.loc[cleaned_telco_df["tenure"] == 0, "TotalCharges"] = 0

# Extract the rows with a missing TotalCharges
invalid_presence = cleaned_telco_df["TotalCharges"].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

# Verify that there are no longer missing values
print(f"Number of rows where the TotalCharges value is missing: {invalid_presence.sum()}")

Number of rows where the TotalCharges value is missing: 0


Now, we have solved the presence error problem by imputing the value of 0 in `cleaned_telco_df`, which is done since the missing values correspond to customers who have not been charged yet.

## Conclusion

## References